<a href="https://colab.research.google.com/github/lucasfelipecdm/fiap-hackathon-vision-guard/blob/main/fiap_hackathon_visionguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install ultralytics
!pip install opencv-python
!pip install email_validator
!pip install yagmail

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
# Import libraries
import os
import cv2
from ultralytics import YOLO
import yagmail
from email_validator import validate_email, EmailNotValidError

# Mount Google Drive
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


In [9]:
# --- CONFIGURATION ---
data_path = userdata.get('DATA_PATH')
# Path to your dataset (replace with your actual path)
dataset_path = data_path + '/dataset3'  # Example: /content/drive/MyDrive/yolov8_dataset

# Path to your video (replace with your actual path)
video_path = data_path + '/test-video/video-test-1.mp4'  # Example: /content/drive/MyDrive/video.mp4
video_path2 = data_path + '/test-video/video-test-2.mp4'  # Example: /content/drive/MyDrive/video.mp4

# Email configuration (replace with your credentials)
SENDER_EMAIL = userdata.get('GMAIL_EMAIL')  # Your Gmail address
SENDER_PASSWORD = userdata.get('GMAIL_PASSWORD')  # Your Gmail password or App Password (recommended)
RECEIVER_EMAIL = "lucasfelipecdm@hotmail.com" # Recipient email address

# Classes to detect (adjust if needed)
CLASSES_TO_DETECT = ['knife']  # Match your dataset's classes

# --- NEW: Create directory in Google Drive ---
output_folder = data_path + '/results'  # Path to your desired folder
os.makedirs(output_folder, exist_ok=True)  # Create if it doesn't exist

In [10]:
# --- FUNCTIONS ---

def train_model(dataset_path):
    """Trains the YOLOv8 model."""
    model = YOLO('yolov8m.pt')  # You can change to a larger model like yolov8s.pt, yolov8m.pt, etc. for better accuracy if needed
    model.train(data=os.path.join(dataset_path, 'data.yaml'), epochs=25, batch=16, imgsz=640)  # Adjust epochs as needed
    return model

def detect_objects(model, video_path):
    """Detects objects in the video and sends an email if any are found."""
    cap = cv2.VideoCapture(video_path)
    object_detected = False
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                if class_name in CLASSES_TO_DETECT:
                    object_detected = True
                    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get box coordinates
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                    cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)  # Add label
                    # --- NEW: Save the frame ---
                    frame_filename = f"frame_{frame_count}.jpg"  # Unique filename
                    frame_path = os.path.join(output_folder, frame_filename)
                    cv2.imwrite(frame_path, frame)  # Save the frame
                    frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'q' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

    return object_detected


def send_email(object_detected):
    """Sends an email notification."""
    if object_detected:
         try:
            # Validate email addresses
            validated_sender = validate_email(SENDER_EMAIL)
            validated_receiver = validate_email(RECEIVER_EMAIL)

            # Use validated and normalized email addresses
            sender_email = validated_sender["email"]
            receiver_email = validated_receiver["email"]

            yag = yagmail.SMTP(user=sender_email, password=SENDER_PASSWORD)
            contents = ["Sharps objects detected in the video!"]  # Email content
            yag.send(to=receiver_email, subject="Sharps Object Detection Alert", contents=contents)
            print("Email sent successfully!")

         except EmailNotValidError as e:
            print(f"Invalid email address: {e}")
         except Exception as e:
             print(f"Error sending email: {e}")
    else:
        print("No sharps objects detected.")

In [11]:
# --- MAIN EXECUTION ---

# 1. Train the model (uncomment if you need to retrain)
model = train_model(dataset_path)
model.save(os.path.join(dataset_path, 'best.pt')) # Save the trained model

# 2. Load the trained model
model = YOLO(os.path.join(dataset_path, 'best.pt'))  # Load the trained model. Make sure the 'best.pt' file is in the correct directory.

# 3. Detect objects in the video
object_detected = detect_objects(model, video_path2)

# print(object_detected)
# 4. Send email notification
# send_email(object_detected)

print("Finished.")

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Colab Notebooks/fiap-hackathon-visionguard-content/dataset3/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=

train: Scanning /content/drive/MyDrive/Colab Notebooks/fiap-hackathon-visionguard-content/dataset3/train/labels... 6130 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6130/6130 [01:53<00:00, 53.83it/s]


train: New cache created: /content/drive/MyDrive/Colab Notebooks/fiap-hackathon-visionguard-content/dataset3/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 353, len(boxes) = 6352. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/Colab Notebooks/fiap-hackathon-visionguard-content/dataset3/valid/labels... 520 images, 0 backgrounds, 0 corrupt: 100%|██████████| 520/520 [00:10<00:00, 47.89it/s] 


val: New cache created: /content/drive/MyDrive/Colab Notebooks/fiap-hackathon-visionguard-content/dataset3/valid/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 100, len(boxes) = 585. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      25/25      7.74G      1.014     0.7567      1.393          2        640: 100%|██████████| 384/384 [01:56<00:00,  3.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:04<00:00,  3.63it/s]

                   all        520        585      0.634      0.749      0.745      0.474



25 epochs completed in 0.859 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.0MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  3.30it/s]


                   all        520        585      0.663      0.755      0.756      0.474
                 knife        520        585      0.663      0.755      0.756      0.474
Speed: 0.2ms preprocess, 5.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train3

0: 384x640 (no detections), 15.7ms
Speed: 1.8ms preprocess, 15.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.8ms
Speed: 2.1ms preprocess, 9.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.9ms
Speed: 2.0ms preprocess, 9.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 2.1ms preprocess, 10.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 knife, 10.0ms
Speed: 1.9ms preprocess, 10.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.2ms
Speed: 1.8ms pr